In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
# 指定したパスからディレクトリ情報のみ抜き出して出力する。
import os
path = '/content/drive/MyDrive/Colab Notebooks/CelebA_8class/data/train'
files = os.listdir(path=path)
folder = [f for f in files if os.path.isdir(os.path.join(path, f))]
print(folder)


['NakedEye_Woman_NoHat', 'NakedEye_Woman_Hat', 'NakedEye_Man_Hat', 'NakedEye_Man_NoHat', 'Glass_Woman_NoHat', 'Glass_Woman_Hat', 'Glass_Man_NoHat', 'Glass_Man_Hat']


In [29]:
import json
# ラベル用リストの作成
path = '/content/drive/MyDrive/Colab Notebooks/CelebA_8class/data/train'
files = os.listdir(path=path)
FolderList = [f for f in files if os.path.isdir(os.path.join(path, f))]
print(FolderList)

ILSVRC_class_index = json.load(open('/content/drive/MyDrive/Colab Notebooks/CelebA_8class/data/imagenet_class_index.json', 'r'))
print(ILSVRC_class_index)
print(ILSVRC_class_index['0'][1])

for folder in ILSVRC_class_index:
  print(ILSVRC_class_index[str(folder)][1])


['NakedEye_Woman_NoHat', 'NakedEye_Woman_Hat', 'NakedEye_Man_Hat', 'NakedEye_Man_NoHat', 'Glass_Woman_NoHat', 'Glass_Woman_Hat', 'Glass_Man_NoHat', 'Glass_Man_Hat']
{'0': ['n01440764', 'Glass_Man_Hat'], '1': ['n01443537', 'Glass_Man_NoHat'], '2': ['n01484850', 'Glass_Woman_Hat'], '3': ['n01491361', 'Glass_Woman_NoHat'], '4': ['n01494475', 'NakedEye_Man_Hat'], '5': ['n01496331', 'NakedEye_Man_NoHat'], '6': ['n01498041', 'NakedEye_Woman_Hat'], '7': ['n01514668', 'NakedEye_Woman_NoHat']}
Glass_Man_Hat
Glass_Man_Hat
Glass_Man_NoHat
Glass_Woman_Hat
Glass_Woman_NoHat
NakedEye_Man_Hat
NakedEye_Man_NoHat
NakedEye_Woman_Hat
NakedEye_Woman_NoHat


VGG16学習済モデルでCelebAを使用したファインチューニング（8クラス）

In [30]:
# パッケージのimport
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms
import torchvision.datasets as datasets

In [31]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

Datasetを作成

In [32]:
# 入力画像の前処理をするクラス
# 訓練時と推論時で処理が異なる


class ImageTransform():
    """
    画像の前処理クラス。訓練時、検証時で異なる動作をする。
    画像のサイズをリサイズし、色を標準化する。
    訓練時はRandomResizedCropとRandomHorizontalFlipでデータオーギュメンテーションする。


    Attributes
    ----------
    resize : int
        リサイズ先の画像の大きさ。
    mean : (R, G, B)
        各色チャネルの平均値。
    std : (R, G, B)
        各色チャネルの標準偏差。
    """

    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(
                    resize, scale=(0.5, 1.0)),  # データオーギュメンテーション
                transforms.RandomHorizontalFlip(),  # データオーギュメンテーション
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
            ]),
            'val': transforms.Compose([
                transforms.Resize(resize),  # リサイズ
                transforms.CenterCrop(resize),  # 画像中央をresize×resizeで切り取り
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
            ])
        }

    def __call__(self, img, phase='train'):
        """
        Parameters
        ----------
        phase : 'train' or 'val'
            前処理のモードを指定。
        """
        return self.data_transform[phase](img)


In [33]:
# セレブ画像へのファイルパスのリストを作成する
def make_datapath_list(phase="train"):
    """
    データのパスを格納したリストを作成する。

    Parameters
    ----------
    phase : 'train' or 'val'
        訓練データか検証データかを指定する

    Returns
    -------
    path_list : list
        データへのパスを格納したリスト
    """

    rootpath = '/content/drive/MyDrive/Colab Notebooks/CelebA_8class/data/'
    target_path = osp.join(rootpath + phase + '/**/*.png')
    print(target_path)

    path_list = []  # ここに格納する

    # globを利用してサブディレクトリまでファイルパスを取得する
    for path in glob.glob(target_path):
        path_list.append(path)

    return path_list

# 実行
train_list = make_datapath_list(phase="train")
val_list = make_datapath_list(phase="val")

train_list

/content/drive/MyDrive/Colab Notebooks/CelebA_8class/data/train/**/*.png
/content/drive/MyDrive/Colab Notebooks/CelebA_8class/data/val/**/*.png


['/content/drive/MyDrive/Colab Notebooks/CelebA_8class/data/train/NakedEye_Woman_NoHat/000001.png',
 '/content/drive/MyDrive/Colab Notebooks/CelebA_8class/data/train/NakedEye_Woman_NoHat/000004.png',
 '/content/drive/MyDrive/Colab Notebooks/CelebA_8class/data/train/NakedEye_Woman_NoHat/000058.png',
 '/content/drive/MyDrive/Colab Notebooks/CelebA_8class/data/train/NakedEye_Woman_NoHat/000071.png',
 '/content/drive/MyDrive/Colab Notebooks/CelebA_8class/data/train/NakedEye_Woman_NoHat/000039.png',
 '/content/drive/MyDrive/Colab Notebooks/CelebA_8class/data/train/NakedEye_Woman_NoHat/000075.png',
 '/content/drive/MyDrive/Colab Notebooks/CelebA_8class/data/train/NakedEye_Woman_NoHat/000045.png',
 '/content/drive/MyDrive/Colab Notebooks/CelebA_8class/data/train/NakedEye_Woman_NoHat/000087.png',
 '/content/drive/MyDrive/Colab Notebooks/CelebA_8class/data/train/NakedEye_Woman_NoHat/000066.png',
 '/content/drive/MyDrive/Colab Notebooks/CelebA_8class/data/train/NakedEye_Woman_NoHat/000019.png',


In [34]:
def PathToLabel(label):
  i = 0
  for index in ILSVRC_class_index:
    if label == (ILSVRC_class_index[str(index)][1]):
      return i
    i += 1
  
  return -1
  
  '''
  for folder in ILSVRC_class_index:
    if folder[1] == label:
      return i
  '''

In [35]:
# セレブの画像のDatasetを作成する
class CelebDataset(data.Dataset):
    """
    アリとハチの画像のDatasetクラス。PyTorchのDatasetクラスを継承。

    Attributes
    ----------
    file_list : リスト
        画像のパスを格納したリスト
    transform : object
        前処理クラスのインスタンス
    phase : 'train' or 'test'
        学習か訓練かを設定する。
    """

    def __init__(self, file_list, transform=None, phase='train'):
        self.file_list = file_list  # ファイルパスのリスト
        self.transform = transform  # 前処理クラスのインスタンス
        self.phase = phase  # train or valの指定

    def __len__(self):
        '''画像の枚数を返す'''
        return len(self.file_list)

    def __getitem__(self, index):
        '''
        前処理をした画像のTensor形式のデータとラベルを取得
        '''

        # index番目の画像をロード
        img_path = self.file_list[index]
        img = Image.open(img_path)  # [高さ][幅][色RGB]

        # 画像の前処理を実施
        img_transformed = self.transform(
            img, self.phase)  # torch.Size([3, 224, 224])

        # 画像のラベルをファイル名から抜き出す
        label = osp.basename(osp.dirname(img_path))
        # ラベルを数値に変更する
        label = PathToLabel(label)

        return img_transformed, label

# 実行
train_dataset = CelebDataset(
    file_list=train_list, transform=ImageTransform(size, mean, std), phase='train')

val_dataset = CelebDataset(
    file_list=val_list, transform=ImageTransform(size, mean, std), phase='val')

# 動作確認
index = 0
print(train_dataset.__getitem__(index)[0].size())
print(train_dataset.__getitem__(index)[1])


torch.Size([3, 224, 224])
7


DataLoaderを作成

In [36]:
# ミニバッチのサイズを指定
batch_size = 32

# DataLoaderを作成
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

# 辞書型変数にまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

# 動作確認
batch_iterator = iter(dataloaders_dict["train"])  # イテレータに変換
inputs, labels = next(
    batch_iterator)  # 1番目の要素を取り出す
print(inputs.size())
print(labels)

torch.Size([32, 3, 224, 224])
tensor([7, 1, 6, 1, 4, 3, 5, 1, 4, 0, 1, 7, 5, 0, 1, 7, 5, 4, 4, 6, 3, 7, 4, 6,
        7, 6, 6, 5, 6, 1, 0, 4])


ネットワークモデルの作成

In [37]:
# 学習済みのVGG-16モデルをロード
# VGG-16モデルのインスタンスを生成
use_pretrained = True  # 学習済みのパラメータを使用
net = models.vgg16(pretrained=use_pretrained)

# デバイスの選択（PyTorchがGPUを使用できるのであれば、GPUを使用するように）
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(torch.cuda.is_available())

# VGG16の最後の出力層の出力ユニットを8つに付け替える
net.classifier[6] = nn.Linear(in_features=4096, out_features=8)
net = net.to(device)  # GPUに転送

# 訓練モードに設定
net.train()

print('ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました')


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


True
ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました


損失関数を定義

In [38]:
# 損失関数の設定
criterion = nn.CrossEntropyLoss()

最適化手法の設定

In [39]:
# 転移学習で学習させるパラメータを、変数params_to_updateに格納する
params_to_update = []

# 学習させるパラメータ名
update_param_names = ["classifier.6.weight", "classifier.6.bias"]

# 学習させるパラメータ以外は勾配計算をなくし、変化しないように設定
for name, param in net.named_parameters():
    if name in update_param_names:
        param.requires_grad = True
        params_to_update.append(param)
        print(name)
    else:
        param.requires_grad = False

# params_to_updateの中身を確認
print("-----------")
print(params_to_update)


classifier.6.weight
classifier.6.bias
-----------
[Parameter containing:
tensor([[-0.0108, -0.0026,  0.0099,  ...,  0.0066, -0.0093,  0.0042],
        [-0.0131,  0.0002, -0.0092,  ..., -0.0025, -0.0018, -0.0049],
        [-0.0052, -0.0028, -0.0026,  ...,  0.0127,  0.0138, -0.0048],
        ...,
        [-0.0038, -0.0124, -0.0134,  ...,  0.0090, -0.0152, -0.0145],
        [ 0.0070,  0.0054,  0.0107,  ...,  0.0040,  0.0070, -0.0049],
        [-0.0067, -0.0081, -0.0146,  ..., -0.0074,  0.0096,  0.0029]],
       requires_grad=True), Parameter containing:
tensor([ 0.0076,  0.0082, -0.0145,  0.0117, -0.0093, -0.0045,  0.0065, -0.0131],
       requires_grad=True)]


In [40]:
# 最適化手法の設定
optimizer = optim.SGD(params=params_to_update, lr=0.001, momentum=0.9)

学習と検証の実施

In [41]:
# モデルを学習させる関数を作成
def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # epochのループ
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの学習と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数

            # 未学習時の検証性能を確かめるため、epoch=0の訓練は省略
            if (epoch == 0) and (phase == 'train'):
                continue

            # データローダーからミニバッチを取り出すループ
            for inputs, labels in tqdm(dataloaders_dict[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)  # 損失を計算
                    _, preds = torch.max(outputs, 1)  # ラベルを予測
                    
  
                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # イタレーション結果の計算
                    # lossの合計を更新
                    epoch_loss += loss.item() * inputs.size(0)  
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率を表示
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))


In [42]:
# 学習・検証を実行する
num_epochs=3
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

Epoch 1/3
-------------


  0%|          | 0/4 [00:00<?, ?it/s]


RuntimeError: ignored

In [ ]:
# PyTorchのネットワークパラメータの保存
save_path = '/content/drive/MyDrive/Colab Notebooks/CelebA_8class/data/weights_teni.pth'

torch.save(net.state_dict(), save_path)
